In [9]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features

df_train = pd.read_csv('./data/train_filtrado.csv')
df_train['precio'] = np.log(df_train['precio'])
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

# df_train, df_test = features_de_csvs(df_train, df_test)

# df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

In [10]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

## LightGBM model

In [11]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.08363501292068126)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.8988911725316586,
 'bagging_freq': 22.0,
 'feature_fraction': 0.6622442122619671,
 'learning_rate': 0.16422725363286422,
 'max_depth': 22.0,
 'num_leaves': 180.0,
 'test_size': 0.20892455926004772}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

## Keras model

In [12]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

def keras_modelo():    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)
    return model

keras_m = KerasRegressor(build_fn=keras_modelo, epochs=10)

## XGBoost

In [13]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        return super(xgb.XGBRegressor, self).fit(x, y, early_stopping_rounds=2, eval_metric='mae', eval_set=[(x, y)])
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)


hps = {'alpha': 20.91434940058063,
       'colsample_bytree': 0.65,
       'learning_rate': 0.14,
       'max_depth': int(16.0),
       'n_estimators': int(150.0),
       'test_size': 0.2,
       'early_stopping_rounds': 5,
       'n_jobs': 4}


n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

xgb_m = XGBoostWrapper(**hps)

## Stacking

In [14]:
from sklearn.preprocessing import MinMaxScaler

from vecstack import StackingTransformer

features = ['habitaciones', 
            'garages', 
            'banos',
            'antiguedad',
            'metroscubiertos', 
            'metrostotales',
            'lat_norm', 'lng_norm'
            'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
                 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'count_id_zona', 'count_ciudad', 'puntaje', 
                     'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen',
                 'count_id_zona'
                 'dias_desde_datos',
                 'meses_desde_datos',
                 'porcentaje_metros',
                 'promedio_precio_hbg_tipo_propiedad']

features += features_test

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

modelos = [('lightgbm', lgb_m), 
#            ('keras', keras_m), 
           ('xgboost', xgb_m)]

stack = StackingTransformer(modelos, regression=True, verbose=2)

stack = stack.fit(x_train, y_train)

s_train = stack.transform(x_train)
s_test = stack.transform(x_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [lightgbm: LightGBMWrapper]
    fold  0:  [0.19816413]
    fold  1:  [0.19671501]
    fold  2:  [0.19689668]
    fold  3:  [0.19677309]
    ----
    MEAN:     [0.19713723] + [0.00059650]

estimator  1: [xgboost: XGBoostWrapper]
[16:11:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:11.9622
Will train until validation_0-mae hasn't improved in 2 rounds.
[1]	validation_0-mae:10.2878
[2]	validation_0-mae:8.84783
[3]	validation_0-mae:7.60936
[4]	validation_0-mae:6.54425
[5]	validation_0-mae:5.62825
[6]	validation_0-mae:4.84051
[7]	validation_0-mae:4.16301
[8]	validation_0-mae:3.5804
[9]	validation_0-mae:3.07941
[10]	validation_0-mae:2.64855
[11]	validation_0-mae:2.27806
[12]	validation_0-mae:1.95949
[13]	validation_0-mae:1.68566
[14]	validation_0-mae:1.45036
[15]	validation_0-mae:1.24828

[92]	validation_0-mae:0.064766
[93]	validation_0-mae:0.064206
[94]	validation_0-mae:0.063814
[95]	validation_0-mae:0.063618
[96]	validation_0-mae:0.063009
[97]	validation_0-mae:0.062803
[98]	validation_0-mae:0.062441
[99]	validation_0-mae:0.06176
[100]	validation_0-mae:0.061275
[101]	validation_0-mae:0.060987
[102]	validation_0-mae:0.060652
[103]	validation_0-mae:0.060446
[104]	validation_0-mae:0.059618
[105]	validation_0-mae:0.059303
[106]	validation_0-mae:0.059052
[107]	validation_0-mae:0.058516
[108]	validation_0-mae:0.057942
[109]	validation_0-mae:0.057567
[110]	validation_0-mae:0.057369
[111]	validation_0-mae:0.056871
[112]	validation_0-mae:0.056793
[113]	validation_0-mae:0.055902
[114]	validation_0-mae:0.055655
[115]	validation_0-mae:0.055121
[116]	validation_0-mae:0.055049
[117]	validation_0-mae:0.054822
[118]	validation_0-mae:0.054197
[119]	validation_0-mae:0.053884
[120]	validation_0-mae:0.053805
[121]	validation_0-mae:0.053455
[122]	validation_0-mae:0.053004
[123]	validation_

[42]	validation_0-mae:0.116814
[43]	validation_0-mae:0.114086
[44]	validation_0-mae:0.111971
[45]	validation_0-mae:0.110218
[46]	validation_0-mae:0.108589
[47]	validation_0-mae:0.106791
[48]	validation_0-mae:0.104888
[49]	validation_0-mae:0.103163
[50]	validation_0-mae:0.101454
[51]	validation_0-mae:0.100084
[52]	validation_0-mae:0.098202
[53]	validation_0-mae:0.096587
[54]	validation_0-mae:0.095402
[55]	validation_0-mae:0.093613
[56]	validation_0-mae:0.091905
[57]	validation_0-mae:0.090844
[58]	validation_0-mae:0.089486
[59]	validation_0-mae:0.088378
[60]	validation_0-mae:0.086888
[61]	validation_0-mae:0.086217
[62]	validation_0-mae:0.085139
[63]	validation_0-mae:0.083864
[64]	validation_0-mae:0.082948
[65]	validation_0-mae:0.081696
[66]	validation_0-mae:0.081045
[67]	validation_0-mae:0.079947
[68]	validation_0-mae:0.079374
[69]	validation_0-mae:0.078441
[70]	validation_0-mae:0.077534
[71]	validation_0-mae:0.077138
[72]	validation_0-mae:0.076395
[73]	validation_0-mae:0.075936
[74]	val

In [15]:
s_train = stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features))
s_test = stack.transform(utils.filtrar_features(df_test_f, features))

Transforming...

estimator  0: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [25]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

In [26]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [27]:
params_2nd = {'bagging_fraction': 0.8999882607358867,
 'bagging_freq': int(95.0),
 'feature_fraction': 0.2570109385381975,
 'learning_rate': 0.13601832720254403,
 'max_depth': int(26.0),
 'num_leaves': int(175.0),
 'test_size': 0.08363501292068126}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + ['stack01', 'stack02']), df_train['precio'].values)

LightGBMWrapper(bagging_fraction=0.8999882607358867, bagging_freq=95,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.2570109385381975, importance_type='split',
                learning_rate=0.13601832720254403, max_depth=26,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=175,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0,
                test_size=0.08363501292068126)

In [28]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + ['stack01', 'stack02']))
df_test_s['target'] = np.exp(df_test_s['target'])
df_test_s[['id', 'target']].to_csv('respuesta33.csv', index = False)

## Prediccion solo con features de stacking

In [17]:
params_2nd = {'bagging_fraction': 0.8924398062087346,
 'bagging_freq': int(36.0),
 'feature_fraction': 0.16167385124183287,
 'learning_rate': 0.054693418899570134,
 'max_depth': int(4.0),
 'num_leaves': int(93.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features)), df_train_f['precio'].values)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



LightGBMWrapper(bagging_fraction=0.8924398062087346, bagging_freq=36,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.16167385124183287, importance_type='split',
                learning_rate=0.054693418899570134, max_depth=4,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=93,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [24]:
keras_mae_train = utils.MAE(y_train, lgb_m_2nd.predict(stack.transform(x_train)))
keras_mae_test = utils.MAE(y_test, lgb_m_2nd.predict(stack.transform(x_test)))
print(f"MAE Stacking (train): {keras_mae_train:.5f}")
print(f"MAE Stacking (test): {keras_mae_test:.5f}")

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

MAE St

In [26]:
s_test_f = stack.transform(utils.filtrar_features(df_test_f, features))
y_pred_test_f = lgb_m_2nd.predict(s_test_f)
df_test_f['target'] = y_pred_test_f
df_test_f[['id', 'target']].to_csv('respuesta21.csv', index = False)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



In [13]:
features = ['stack01', 'stack02', 'stack03']

def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth = args

    lgb_train = lgb.Dataset(s_train, y_train)
#     lgb_eval = lgb.Dataset(s_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
#                     valid_sets=lgb_eval,
                    num_boost_round=250,
#                     early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(s_test, num_iteration=gbm.best_iteration)
    return utils.MAE(y_test, y_pred_test)

space = [hp.quniform('num_leaves', 30, 130, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 20, 1)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

 12%|█▏        | 12/100 [00:48<05:53,  4.02s/it, best loss: 437714.00137358136]


KeyboardInterrupt: 

In [141]:

# keras_mae_train = utils.MAE(y_test, lgb_m.predict(x_test_s))
# print(f"MAE Keras (train): {keras_mae_train:.5f}")

MAE Keras (train): 524925.45271
